# Install required packages

<span style="color: yellow;">To run training of our models we need to copy this file into google colab and run it there.</span> If you want to use it in other environment, you need to change the code.

In [1]:
!pip install -q ultralytics
!pip install -q roboflow

# Get dataset
https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc/dataset/1/download

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="iC3wOYdGSYwNTKyhSFUu")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(12)
dataset = version.download("yolov5", location="../data/training")

Moving files to another subfolder because otherwise our code will crash.


In [3]:
import shutil
import os

if os.path.exists("../data/training/train") and not os.path.exists("../data/training/data/training/train"):
    shutil.move("../data/training/train", "../data/training/data/training/train")
if os.path.exists("../data/training/valid") and not os.path.exists("../data/training/data/training/valid"):
    shutil.move("../data/training/valid", "../data/training/data/training/valid")
if os.path.exists("../data/training/test") and not os.path.exists("../data/training/data/training/test"):
    shutil.move("../data/training/test", "../data/training/data/training/test")


# Training

First we will create a functions that will help us to train our models

### Train yolo model

In [4]:
import torch
from ultralytics import YOLO


def train_yolo(model_name, epochs, imgsz, train_folder = None, gpu = True):
    """
    This function trains a YOLO model using the specified parameters.

    Parameters:
    model_name (str): The name of the model to be trained.
    epochs (int): The number of training epochs.
    imgsz (int or tuple): The size of the images for training.
    train_folder (str, optional): The folder where the training results will be saved. Defaults to None.
    gpu (bool, optional): Flag to indicate whether to use GPU for training. Defaults to True.

    Raises:
    SystemError: If GPU is not available and gpu is set to True.
    """
    # Check if GPU is available
    if not torch.cuda.is_available() and gpu:
        raise SystemError('GPU device not found')
    yolo = YOLO(model_name)
    yolo.train(data=f'{dataset.location}/data.yaml', epochs=epochs, imgsz=imgsz, project=train_folder, name=model_name, exist_ok=True)
    



### copy trained model weights and all run files to google drive
This function will copy all the files from the run folder to the google drive.

In [5]:
import shutil
import os
try:
    # Sprawdzenie, czy jesteśmy w Google Colab
    from google.colab import drive
    in_colab = True
except ImportError:
    in_colab = False



def copy_to_drive(source_folder, destination_folder):
    if not in_colab:
        raise SystemError('You can copy files to drive only in Google Colab')
    drive.mount('/content/drive')

    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Copy the entire folder recursively
    try:
        shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
        print(f"Successfully copied '{source_folder}' to '{destination_folder}'")
    except Exception as e:
        print(f"Error copying folder: {e}")
    drive.flush_and_unmount()

We want to train and compare 3 different versions of YOLO - yolov5x, yolov8x and yolov11x.

In [ ]:
project_name = "football_analysis"

# models = ["yolov5x.pt", "yolov8x.pt", "yolov11x.pt"]
models = ["yolov5nu.pt"]
epochs = 100
imgsz = 640
train_folder = "../runs/train/" + project_name
results_folder = "/content/drive/MyDrive/Colab/" + project_name

for model in models:
    train_folder = train_folder + "/" + model
    train_yolo(model_name=model, epochs=epochs, imgsz=imgsz, train_folder=train_folder)
    destination_folder = results_folder + "/" + model
    copy_to_drive(source_folder=train_folder, destination_folder=destination_folder)



### Models comparison

We compare the models by their mAP50 and mAP50-95 metrics

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

results = []
models = ["yolo5x", "yolo8x"]
results_folder = "../runs/train/"

for model in models:
    results_file = os.path.join(results_folder, model, "results.csv")
    print(results_file)
    
    if os.path.exists(results_file):
        df = pd.read_csv(results_file)
        
        # Extracting results from the last epoch
        last_epoch = df.iloc[-1]
        mAP50 = last_epoch['metrics/mAP50(B)']
        mAP50_95 = last_epoch['metrics/mAP50-95(B)']
        results.append((model, mAP50, mAP50_95))

 # Prepare data for plotting
labels = ['mAP50', 'mAP50-95']
values = [result[1:] for result in results]  # Extract mAP50 and mAP50-95 values
values = np.array(values)

# Set up the bar width and positions
bar_width = 0.35
x = np.arange(len(labels))

# Create the bar plot
fig, ax = plt.subplots()
for i, (model, mAP50, mAP50_95) in enumerate(results):
        bars = ax.bar(x + i * bar_width, [mAP50, mAP50_95], bar_width, label=model)
        # Display the value on top of the bars
        for bar in bars:
            yval = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 3), ha='center', va='bottom')

# Add labels and title
ax.set_xlabel('Metrics')
ax.set_ylabel('Values')
ax.set_title('Comparison of mAP50 and mAP50-95 for different models')
ax.set_xticks(x + bar_width / 2)
ax.set_xticklabels(labels)
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()
